In [1]:
import struct
import numpy as np

In [2]:
def load_images(filename):
    with open(filename, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows, cols)
    return images

def load_labels(filename):
    with open(filename, 'rb') as f:
        magic, num_labels = struct.unpack('>II', f.read(8))
        labels = np.fromfile(f, dtype=np.uint8)
    return labels

train_images = 'data/train-images-idx3-ubyte/train-images-idx3-ubyte'
train_labels = 'data/train-labels-idx1-ubyte/train-labels-idx1-ubyte'
test_images = 'data/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte'
test_labels = 'data/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte'

x_train = load_images(train_images)
y_train = load_labels(train_labels)
x_test = load_images(test_images)
y_test = load_labels(test_labels)

In [3]:
# Activation functions
class ReLU():
    def reLU(self, x):
        return np.maximum(0, x)
    def reLU_prime(self, x):
        return x > 0
    def forward(self, input):
        self.input = input
        return self.reLU(self.input)
    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.reLU(self.input))

class Sigmoid():
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    def sigmoid_prime(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)
    def forward(self, input):
        self.input = input
        return self.sigmoid(self.input)
    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.sigmoid_prime(self.input))

class Tanh():
    def tanh(self, x):
        return np.tanh(x)
    def tanh_prime(self, x):
        return 1 - np.tanh(x) ** 2
    def forward(self, input):
        self.input = input
        return self.tanh(self.input)
    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.tanh_prime(self.input))
    
# Loss functions
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)
def mse_derivative(y_true, y_pred):
    return 2 * (y_pred - y_true) / len(y_true)

def bce(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
def bce_derivative(y_true, y_pred):
    return (y_pred - y_true) / (y_pred * (1 - y_pred)) / len(y_true)

def cce(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
def cce_derivative(y_true, y_pred):
    return (y_pred - y_true) / len(y_true)

def hinge(y_true, y_pred):
    return np.mean(np.maximum(0, 1 - y_true * y_pred))
def hinge_derivative(y_true, y_pred):
    return -y_true * (y_true * y_pred < 1) / len(y_true)

In [4]:
class Dense():
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

class Reshape():
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

class Convolutional():
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)
    
    def correlate2d(self, input, kernel, mode='valid'):
        input_height, input_width = input.shape
        kernel_height, kernel_width = kernel.shape

        if mode == 'valid':
            output_height = input_height - kernel_height + 1
            output_width = input_width - kernel_width + 1
        elif mode == 'same':
            output_height = input_height
            output_width = input_width
        elif mode == 'full':
            output_height = input_height + kernel_height - 1
            output_width = input_width + kernel_width - 1
        else:
            raise ValueError("Invalid mode. Mode must be one of 'valid', 'same', or 'full'.")

        correlation = np.zeros((output_height, output_width))
        for i in range(output_height):
            for j in range(output_width):
                input_patch = input[i:i+kernel_height, j:j+kernel_width]
                correlation[i, j] = np.sum(input_patch * kernel)
        return correlation

    def convolve2d(self, input, kernel, mode='full'):
        input_height, input_width = input.shape
        kernel_height, kernel_width = kernel.shape

        if mode == 'full':
            output_height = input_height + kernel_height - 1
            output_width = input_width + kernel_width - 1
            padding = (kernel_height - 1, kernel_width - 1)
            input = np.pad(input, padding, mode='constant')
        elif mode == 'valid':
            output_height = input_height - kernel_height + 1
            output_width = input_width - kernel_width + 1
        elif mode == 'same':
            output_height = input_height
            output_width = input_width
            padding_height = (kernel_height - 1) // 2
            padding_width = (kernel_width - 1) // 2
            padding = ((padding_height, padding_height), (padding_width, padding_width))
            input = np.pad(input, padding, mode='constant')
        else:
            raise ValueError("Invalid mode. Mode must be one of 'full', 'valid', or 'same'.")

        convolution = np.zeros((output_height, output_width))
        for i in range(output_height):
            for j in range(output_width):
                input_patch = input[i:i+kernel_height, j:j+kernel_width]
                convolution[i, j] = np.sum(input_patch * kernel)
        return convolution
    
    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += self.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)
        
        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = self.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += self.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient

In [5]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 1000, learning_rate = 0.01, verbose = True):
    for i in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            output = predict(network, x)
            error += loss(y, output)
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)

        error /= len(x_train)
        if verbose:
            print(f"{i + 1}/{epochs}, error={error}")

In [6]:
def preprocess_data(x, y, limit):
    # Select limit samples for each class
    indices = []
    for i in range(10):
        class_indices = np.where(y == i)[0][:limit]
        indices.extend(class_indices)
    
    # Shuffle the indices
    np.random.shuffle(indices)
    
    # Select the samples corresponding to the indices
    x = x[indices]
    y = y[indices]
    
    # Reshape and normalize input data
    x = x.reshape(len(x), 1, 28, 28).astype("float32") / 255
    
    # One-hot encode the output labels
    num_classes = 10
    y_encoded = np.zeros((len(y), num_classes))
    for i in range(len(y)):
        y_encoded[i, y[i]] = 1
    y_encoded = y_encoded.reshape(len(y), num_classes, 1)
    
    return x, y_encoded

x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 10)

In [7]:
# neural network - with convolution
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 10),
    Sigmoid()
]

# train
train(
    network,
    bce,
    bce_derivative,
    x_train,
    y_train,
    epochs=5,
    learning_rate=0.1
)

1/5, error=0.3778852032955341
2/5, error=0.3032083847218757
3/5, error=0.2734398492998715
4/5, error=0.24652819121480007
5/5, error=0.22073805842606242


In [12]:
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    print('predicted:', np.argmax(output), '\ttrue:', np.argmax(y))


predicted: 5 	true: 5
predicted: 6 	true: 6
predicted: 9 	true: 4
predicted: 9 	true: 8
predicted: 0 	true: 0
predicted: 8 	true: 8
predicted: 9 	true: 9
predicted: 2 	true: 3
predicted: 8 	true: 5
predicted: 1 	true: 1
predicted: 1 	true: 1
predicted: 2 	true: 2
predicted: 7 	true: 7
predicted: 9 	true: 5
predicted: 8 	true: 3
predicted: 8 	true: 3
predicted: 7 	true: 7
predicted: 5 	true: 0
predicted: 7 	true: 7
predicted: 0 	true: 0
predicted: 1 	true: 1
predicted: 6 	true: 2
predicted: 5 	true: 9
predicted: 7 	true: 5
predicted: 7 	true: 0
predicted: 7 	true: 7
predicted: 5 	true: 0
predicted: 3 	true: 3
predicted: 7 	true: 9
predicted: 9 	true: 9
predicted: 1 	true: 1
predicted: 1 	true: 7
predicted: 8 	true: 8
predicted: 8 	true: 5
predicted: 6 	true: 3
predicted: 0 	true: 0
predicted: 7 	true: 0
predicted: 7 	true: 9
predicted: 8 	true: 8
predicted: 1 	true: 1
predicted: 2 	true: 0
predicted: 2 	true: 2
predicted: 9 	true: 6
predicted: 3 	true: 3
predicted: 7 	true: 7
predicted: